In [135]:
from llama_index import SimpleDirectoryReader
from llama_index.text_splitter import SentenceSplitter
from llama_index.schema import TextNode
from llama_index import VectorStoreIndex
from llama_index import Document
from llama_index.vector_stores import AstraDBVectorStore
import astrapy
import re 
import requests
import os
import json
import datetime

import dotenv
from dotenv import load_dotenv

print('hello')

hello


In [124]:
file_path = '/Users/mehulkhetrapal/Documents/GitHub/Midas/midas/transcriptdata'

load = SimpleDirectoryReader(file_path).load_data()
text_data = []
metadata = []
for i in range(0,len(load)):
     text_data.append(load[i].text)
     title_split = load[i].metadata['file_name'].split('__')
     conversation_id = title_split[0]
     datetime1 = title_split[1].split('.')[0]
     datetime_object = datetime.datetime.strptime(datetime1, "%Y-%m-%d_%I-%M-%p")
     date_string = datetime_object.strftime("%m/%d/%Y, %H:%M")
     metadata.append({'convo_id': conversation_id, 'date': date_string})

text_with_metadata = [(text, meta) for text, meta in zip(text_data, metadata)]
text_with_metadata

[("Call 1\n\nHannah: Hello, Kevin! This is Hannah from Tommy Bahama. How are you today?\n\nKevin: Hi Hannah, I'm doing well, thank you. How about yourself?\n\nHannah: I'm great, thanks for asking. Now, how can I assist you today?\n\nKevin: We're looking to get some custom-made jackets for our team here at LlamaIndex. I heard Tommy Bahama does quality work, so I thought I'd reach out.\n\nHannah: Absolutely, Kevin! We'd be thrilled to help you out. Before we dive in, can you give me an idea of the quantity you're looking for?\n\nKevin: We're thinking about 500 jackets.\n\nHannah: Fantastic! That's a significant order. Let's talk about what you're looking for specifically, and we can go from there. Are there any specific design elements or features you have in mind?\n\nKevin: Well, first off, I'm curious about your pricing. Can you give me a ballpark figure for custom jackets?\n\nHannah: Of course, Kevin. Our pricing depends on a few factors like the materials, customization options, and 

In [125]:
text_splitter = SentenceSplitter()
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
chunk_ids = []
for doc_idx, doc in enumerate(text_with_metadata):
    cur_text_chunks = text_splitter.split_text(doc[0])
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))
    chunk_ids.extend(list(range(len(cur_text_chunks))))

In [127]:
from llama_index.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = text_with_metadata[doc_idxs[idx]]
    chunk_id = chunk_ids[idx]
    metadata = src_doc[1]
    metadata['chunk_id'] = chunk_id
    node.metadata = metadata
    nodes.append(node)
nodes


[TextNode(id_='9e3629b5-4082-43c7-b8ec-8f07571b7726', embedding=None, metadata={'convo_id': '1', 'date': '02/03/2024, 19:28', 'chunk_id': 0}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Call 1\n\nHannah: Hello, Kevin! This is Hannah from Tommy Bahama. How are you today?\n\nKevin: Hi Hannah, I'm doing well, thank you. How about yourself?\n\nHannah: I'm great, thanks for asking. Now, how can I assist you today?\n\nKevin: We're looking to get some custom-made jackets for our team here at LlamaIndex. I heard Tommy Bahama does quality work, so I thought I'd reach out.\n\nHannah: Absolutely, Kevin! We'd be thrilled to help you out. Before we dive in, can you give me an idea of the quantity you're looking for?\n\nKevin: We're thinking about 500 jackets.\n\nHannah: Fantastic! That's a significant order. Let's talk about what you're looking for specifically, and we can go from there. Are there any specific design elements or features you have in mind?

"Call 1\n\nHannah: Hello, Kevin! This is Hannah from Tommy Bahama. How are you today?\n\nKevin: Hi Hannah, I'm doing well, thank you. How about yourself?\n\nHannah: I'm great, thanks for asking. Now, how can I assist you today?\n\nKevin: We're looking to get some custom-made jackets for our team here at LlamaIndex. I heard Tommy Bahama does quality work, so I thought I'd reach out.\n\nHannah: Absolutely, Kevin! We'd be thrilled to help you out. Before we dive in, can you give me an idea of the quantity you're looking for?\n\nKevin: We're thinking about 500 jackets.\n\nHannah: Fantastic! That's a significant order. Let's talk about what you're looking for specifically, and we can go from there. Are there any specific design elements or features you have in mind?\n\nKevin: Well, first off, I'm curious about your pricing. Can you give me a ballpark figure for custom jackets?\n\nHannah: Of course, Kevin. Our pricing depends on a few factors like the materials, customization options, and qu

In [130]:
upload_text = [i.text for i in nodes]

headers = {"Authorization": f"Bearer {os.getenv('HUGGINGFACE_API_TOKEN')}"}
embedding_endpoint = "https://api-inference.huggingface.co/pipeline/feature-extraction/BAAI/bge-small-en-v1.5"
embedding_payload = {
        "inputs": upload_text,
        "options": {
        "wait_for_model": True
        }
    }
embedding_response = requests.post(embedding_endpoint, headers=headers, json=embedding_payload)
embeddings = list(json.loads(embedding_response.text))

for node, embedding in zip(nodes, embeddings):
    node.embedding = embedding

In [131]:
nodes

[TextNode(id_='9e3629b5-4082-43c7-b8ec-8f07571b7726', embedding=[-0.05157437175512314, -0.0025211547035723925, -0.002185012912377715, -0.010861820541322231, 0.061315495520830154, 0.02801850065588951, 0.045600928366184235, 0.017398521304130554, -0.058174487203359604, -0.02242056094110012, 0.030224120244383812, 0.009623592719435692, -0.012907274067401886, -0.009770313277840614, 0.11002042144536972, 0.04032227769494057, 0.03305237740278244, -0.05874980241060257, -0.10602003335952759, 0.04047763720154762, 0.03642937168478966, -0.068758025765419, -0.014026597142219543, 0.0439765639603138, 0.0060877446085214615, -0.0488012358546257, 0.005891911219805479, 0.006838101893663406, 0.004970175679773092, -0.18495118618011475, 0.05274967849254608, -0.021730316802859306, 0.039825692772865295, -0.03949592635035515, 0.005662173964083195, 0.024309925734996796, -0.01663193479180336, 0.00027309125289320946, 0.03797883167862892, 0.04110312834382057, -0.011074901558458805, 0.02179875411093235, -0.0750997215

In [132]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [133]:
os.getenv("ASTRA_DB_APPLICATION_TOKEN")

'AstraCS:spDQvAJdTGyfdjNAwAQvXcuy:8a639c244903adff2f51a8dd6fd4a82a4f3dd577e3ac1d5a69bc1f9c346dda33'

In [134]:
vector_store = AstraDBVectorStore(
            token=os.getenv('ASTRA_DB_APPLICATION_TOKEN'),
            api_endpoint=os.getenv('ASTRA_DB_API_ENDPOINT'),
            collection_name="midas_collection",
            embedding_dimension=384,
            namespace = 'transcripts'
)

vector_store.add(nodes)

['9e3629b5-4082-43c7-b8ec-8f07571b7726',
 '01a7a17b-d142-43b3-9b67-43a91c823b3f',
 '92c1b326-eb53-4a2c-9027-90b86f08ef8a',
 'ebcfc2be-1c5a-4d71-8006-a15b8be4d9e8',
 'e1922942-3f71-4961-88ac-ac32f3ca1f1a',
 '02d06645-cd7d-411a-a236-3b59b8a3992e',
 '6ad555ec-a830-482d-966a-a817a3a9db6c',
 '3013b3f0-4556-4a0c-a7a0-59cee4a11014',
 '644f97b8-6e88-4cc2-86f2-79aebe048e44',
 '0f1da70b-9574-4448-b20f-551d22078770']